In [2]:
from bs4 import BeautifulSoup    
import requests  
import re
import time
import pandas as pd
import numpy as np
import itertools
from joblib import Parallel, delayed
from tqdm.notebook import tqdm




# Функция по извлечению данных со страницы объявления в словарь data_dict
def parsing_page_one_ad(url):
    
    response = requests.get(url)
    response.encoding ='utf8'   
    
    # Теперь создадим объект BeautifulSoup    
    page = BeautifulSoup(response.text, 'html.parser')
    data_dict = {}

    data_dict['car_url'] = url
    data_dict['parsing_unixtime'] = int(time.time())


     # в разделах script ищем вхождение 'complectation":{"id"'
    for script in page.find_all("script"):
        if 'complectation":{"id"' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
    # в a  ищем 'complectation":' и содержимое между {}. Отсекаем начало, оставляя только содержимое словаря
            data_dict['complectation_dict'] = re.search(r'complectation":{"id.*?}', a)[0][15:]

        if 'equipment":{' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
        # в a  ищем 'equipment":' и содержимое между {}. Отсекаем начало, оставляя только содержимое словаря
            data_dict['equipment_dict'] = re.search(r'equipment":{.*?}', a)[0][11:]

        if '{"mileage":' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
        # в a  ищем '{"mileage":' и содержимое между {}. Отсекаем начало, оставляя только содержимое словаря
            data_dict['mileage'] = re.search(r'"mileage":\d*', a)[0][10:]

        if '"model_info":' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
            data_dict['model_info'] = re.search(r'"model_info":{.*?}', a)[0][13:]
            data_dict['model_name'] = re.search(r'model_info":{"code":".*?"', a)[0][20:].strip('"')

        if 'super_gen":{' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
            data_dict['super_gen'] = re.search(r'super_gen":{.*?}', a)[0][11:] 

        if 'vendor":"' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
            data_dict['vendor'] = re.search(r'vendor":".*?"', a)[0][9:].strip('"')

    for tag in page.find_all('h1'):
        data_dict['title'] = tag.text

    for tag in page.find_all('span', class_ = "OfferPriceCaption__price"):
        data_dict['price'] = tag.text


    span_CardInfoRow__cell = page.find_all('span', {'class': 'CardInfoRow__cell'})



    for i,tag in enumerate (span_CardInfoRow__cell):
        if tag.text == "Пробег":
            data_dict['Пробег'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел

        if tag.text == "Цвет":
            data_dict['color'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел

        if tag.text == "Двигатель":
            data_dict['Engine'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел

        if tag.text == "год выпуска":
            data_dict['productionDate'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел
            
        if tag.text == "Коробка":
            data_dict['vehicleTransmission'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел
            
        if tag.text == "Кузов":
            data_dict['bodyType'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел
        
        if tag.text == "Владельцы":
            data_dict['Владельцы'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел

        if tag.text == "Владение":
            data_dict['Владение'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел    

        if tag.text == "ПТС":
            data_dict['ПТС'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел    

        if tag.text == "Привод":
            data_dict['Привод'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел 

        if tag.text == "Руль":
            data_dict['Руль'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел

        if tag.text == "Состояние":
            data_dict['Состояние'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел 

        if tag.text == "Таможня":
            data_dict['Таможня'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел

#    time.sleep(0.5)
    return data_dict

    
# Функция по созданию списка ссылок links_list  на объявления о продаже автомобилей  
def extraction_links(url):
    links_list =[] 
#    response = requests.get(url, headers={'User-Agent': UserAgent().chrome})  
    response = requests.get(url)

    response.encoding ='utf8'
    page = BeautifulSoup(response.text, 'html.parser') 
    links = page.find_all('a', class_='Link ListingItemTitle-module__link')
    
    for link in links:
        links_list.append(link.get("href"))
    return links_list

In [3]:
import pandas as pd

df = pd.DataFrame()  # инициализируем датафрейм

url_link_list = []  # инициализируем список страниц по годами и номерами

# указываем года и диапазон страниц, для парсинга. В просмотренных примерах больше 100 страниц не найдено,
# поэтому как граничное значение используем 100.
ranges = [range(1940, 1970), range(1, 10)]

for index in itertools.product(*ranges):
    # формируем ссылки страницы с объявлениями index[0] - year, index[1] - page
    url = (
        f"https://auto.ru/moskva/cars/{index[0]}-year/all/?output_type=table&page={index[1]}")
    print('url', url)
    url_link_list.append(url)  # заносим их в список

links_list = []  # список списков ссылок на объявления c одной страницы таблицы объявлений

# извлекаем в links_list список ссылок на объявления
try:
    links_list = Parallel(n_jobs=2)(delayed(extraction_links)(
        url_links) for url_links in url_link_list)
except:
    pass

ads_dict_list = []  # список словарей содержимого объявлений

for links in links_list:
    try:
        ads_dict_list = Parallel(n_jobs=2)(
            delayed(parsing_page_one_ad)(ad_url) for ad_url in links)
    except:
        pass

    for ad in ads_dict_list:
        try:
            df = df.append(ad, ignore_index=True)
            print('ad', ad)
        except:
            pass    


df.to_csv('_auto_ru_moscow_1940-1970_10_prev.csv', encoding='utf-8',
          index=False)  # записываем содержимое датафрейма в файл

url https://auto.ru/moskva/cars/1940-year/all/?output_type=table&page=1
url https://auto.ru/moskva/cars/1940-year/all/?output_type=table&page=2
url https://auto.ru/moskva/cars/1940-year/all/?output_type=table&page=3
url https://auto.ru/moskva/cars/1940-year/all/?output_type=table&page=4
url https://auto.ru/moskva/cars/1940-year/all/?output_type=table&page=5
url https://auto.ru/moskva/cars/1940-year/all/?output_type=table&page=6
url https://auto.ru/moskva/cars/1940-year/all/?output_type=table&page=7
url https://auto.ru/moskva/cars/1940-year/all/?output_type=table&page=8
url https://auto.ru/moskva/cars/1940-year/all/?output_type=table&page=9
url https://auto.ru/moskva/cars/1941-year/all/?output_type=table&page=1
url https://auto.ru/moskva/cars/1941-year/all/?output_type=table&page=2
url https://auto.ru/moskva/cars/1941-year/all/?output_type=table&page=3
url https://auto.ru/moskva/cars/1941-year/all/?output_type=table&page=4
url https://auto.ru/moskva/cars/1941-year/all/?output_type=table

ad {'car_url': 'https://auto.ru/cars/used/sale/packard/clipper/1098741300-a6372bd9/', 'parsing_unixtime': 1622017367, 'complectation_dict': '{"id":"0"}', 'equipment_dict': '{"condition":true}', 'mileage': '74000', 'model_info': '{"code":"CLIPPER","name":"Clipper","ru_name":"клиппер","morphology":{}', 'model_name': 'CLIPPER', 'super_gen': '{"id":"20660716","year_from":1941,"year_to":1947,"price_segment":"PREMIUM"}', 'vendor': 'AMERICAN', 'title': 'Packard Clipper', 'price': '3\xa0900\xa0000\xa0₽', 'productionDate': '1942', 'Пробег': '74 000 км', 'bodyType': 'седан', 'color': 'чёрный', 'Engine': '4.6 л / 125 л.с. / Бензин', 'vehicleTransmission': 'механическая', 'Привод': 'задний', 'Руль': 'Левый', 'Состояние': 'Не требует ремонта', 'Владельцы': '1 владелец', 'ПТС': 'Оригинал', 'Таможня': 'Растаможен'}
ad {'car_url': 'https://auto.ru/cars/used/sale/gaz/67/1101530820-57276075/', 'parsing_unixtime': 1622017367, 'complectation_dict': '{"id":"0"}', 'equipment_dict': '{}', 'mileage': '10000',

ad {'car_url': 'https://auto.ru/cars/used/sale/ford/custom/1102062882-a8628205/', 'parsing_unixtime': 1622017372, 'complectation_dict': '{"id":"0"}', 'equipment_dict': '{"electro-window-front":true,"light-interior":true,"air-suspension":true,"leather":true}', 'mileage': '64388', 'model_info': '{"code":"CUSTOM","name":"Custom","ru_name":"Кастом","morphology":{}', 'model_name': 'CUSTOM', 'super_gen': '{"id":"20703108","year_from":1949,"year_to":1955,"price_segment":"MEDIUM"}', 'vendor': 'AMERICAN', 'title': 'Ford Custom', 'price': '4\xa0885\xa0000\xa0₽', 'productionDate': '1950', 'Пробег': '64 388 км', 'bodyType': 'купе', 'color': 'пурпурный', 'Engine': '3.7 л / 95 л.с. / Бензин', 'vehicleTransmission': 'механическая', 'Привод': 'задний', 'Руль': 'Левый', 'Состояние': 'Не требует ремонта', 'Владельцы': '1 владелец', 'ПТС': 'Дубликат', 'Таможня': 'Растаможен'}
ad {'car_url': 'https://auto.ru/cars/used/sale/gaz/m_20/1103592191-5b8f6d73/', 'parsing_unixtime': 1622017372, 'complectation_dict

ad {'car_url': 'https://auto.ru/cars/used/sale/gaz/m_20/1098629830-2574bcd8/', 'parsing_unixtime': 1622017377, 'complectation_dict': '{"id":"0"}', 'equipment_dict': '{}', 'mileage': '98385', 'model_info': '{"code":"M_20","name":"М-20 «Победа»","ru_name":"М-20 «Победа»","morphology":{}', 'model_name': 'M_20', 'super_gen': '{"id":"20258133","year_from":1946,"year_to":1958,"price_segment":"MEDIUM"}', 'vendor': 'RUSSIAN', 'title': 'ГАЗ М-20 «Победа»', 'price': '100\xa0000\xa0₽', 'productionDate': '1952', 'Пробег': '98 385 км', 'bodyType': 'фастбек', 'color': 'голубой', 'Engine': '2.1 л / 52 л.с. / Бензин', 'vehicleTransmission': 'механическая', 'Привод': 'задний', 'Руль': 'Левый', 'Состояние': 'Не требует ремонта', 'Владельцы': '1 владелец', 'ПТС': 'Оригинал', 'Таможня': 'Растаможен'}
ad {'car_url': 'https://auto.ru/cars/used/sale/rolls_royce/silver_wraith/1101202795-13a4afd0/', 'parsing_unixtime': 1622017377, 'complectation_dict': '{"id":"0"}', 'equipment_dict': '{}', 'mileage': '95013', 

ad {'car_url': 'https://auto.ru/cars/used/sale/dkw/3_6/1075395609-8e03822c/', 'parsing_unixtime': 1622017382, 'complectation_dict': '{"id":"0"}', 'equipment_dict': '{}', 'mileage': '30000', 'model_info': '{"code":"3_6","name":"3=6","ru_name":"3=6","morphology":{}', 'model_name': '3_6', 'super_gen': '{"id":"21565907","name":"F91","ru_name":"Ф91","year_from":1953,"year_to":1955,"price_segment":"MEDIUM"}', 'vendor': 'EUROPEAN', 'title': 'DKW 3=6 F91', 'price': '1\xa0500\xa0000\xa0₽', 'productionDate': '1954', 'Пробег': '30 000 км', 'bodyType': 'седан 2 дв.', 'color': 'красный', 'Engine': '0.9 л / 34 л.с. / Бензин', 'vehicleTransmission': 'механическая', 'Привод': 'передний', 'Руль': 'Левый', 'Состояние': 'Не требует ремонта', 'Владельцы': '1 владелец', 'ПТС': 'Оригинал', 'Таможня': 'Растаможен'}
ad {'car_url': 'https://auto.ru/cars/used/sale/moscvich/401/1102752524-889b8c45/', 'parsing_unixtime': 1622017382, 'complectation_dict': '{"id":"0"}', 'equipment_dict': '{"bluetooth":true,"aux":tr

...

ad {'car_url': 'https://auto.ru/cars/used/sale/uaz/469/1091756840-07027383/', 'parsing_unixtime': 1622017461, 'complectation_dict': '{"id":"0"}', 'equipment_dict': '{}', 'mileage': '100', 'model_info': '{"code":"469","name":"469","ru_name":"469","morphology":{}', 'model_name': '469', 'super_gen': '{"id":"7869966","year_from":1972,"year_to":2011,"price_segment":"ECONOMY"}', 'vendor': 'RUSSIAN', 'title': 'УАЗ 469  Опытный образец', 'price': '310\xa0000\xa0₽', 'productionDate': '1966', 'Пробег': '100 км', 'bodyType': 'внедорожник открытый', 'color': 'серый', 'Engine': '2.5 л / 75 л.с. / Бензин', 'vehicleTransmission': 'механическая', 'Привод': 'полный', 'Руль': 'Левый', 'Состояние': 'Не требует ремонта', 'Владельцы': '1 владелец', 'ПТС': 'Оригинал', 'Таможня': 'Растаможен'}
ad {'car_url': 'https://auto.ru/cars/used/sale/gaz/21/1102008936-4b4392ec/', 'parsing_unixtime': 1622017462, 'complectation_dict': '{"id":"0"}', 'equipment_dict': '{}', 'mileage': '140000', 'model_info': '{"code":"21",

ad {'car_url': 'https://auto.ru/cars/used/sale/gaz/21/1102718566-c8eef316/', 'parsing_unixtime': 1622017469, 'complectation_dict': '{"id":"0"}', 'equipment_dict': '{}', 'mileage': '1000', 'model_info': '{"code":"21","name":"21 «Волга»","ru_name":"21 «Волга»","morphology":{}', 'model_name': '21', 'super_gen': '{"id":"7867811","year_from":1956,"year_to":1970,"price_segment":"MEDIUM"}', 'vendor': 'RUSSIAN', 'title': 'ГАЗ 21 «Волга»  21Р', 'price': '370\xa0000\xa0₽', 'productionDate': '1967', 'Пробег': '1 000 км', 'bodyType': 'седан', 'color': 'бежевый', 'Engine': '2.5 л / 75 л.с. / Бензин', 'vehicleTransmission': 'механическая', 'Привод': 'задний', 'Руль': 'Левый', 'Состояние': 'Не требует ремонта', 'Владельцы': '2 владельца', 'ПТС': 'Оригинал', 'Владение': '2 года и 7 месяцев', 'Таможня': 'Растаможен'}
ad {'car_url': 'https://auto.ru/cars/used/sale/zaz/965/1103301998-1d8dc372/', 'parsing_unixtime': 1622017469, 'complectation_dict': '{"id":"0"}', 'equipment_dict': '{}', 'mileage': '50000'

ad {'car_url': 'https://auto.ru/cars/used/sale/gaz/69/1103522231-64fd5d83/', 'parsing_unixtime': 1622017475, 'complectation_dict': '{"id":"0"}', 'equipment_dict': '{"steel-wheels":true,"turnbuckle":true,"panorama-roof":true,"leather":true}', 'mileage': '36984', 'model_info': '{"code":"69","name":"69","ru_name":"69","morphology":{}', 'model_name': '69', 'super_gen': '{"id":"20351692","year_from":1953,"year_to":1972,"price_segment":"MEDIUM"}', 'vendor': 'RUSSIAN', 'title': 'ГАЗ 69 69А', 'price': '350\xa0000\xa0₽', 'productionDate': '1968', 'Пробег': '36 984 км', 'bodyType': 'внедорожник 5 дв.', 'color': 'зелёный', 'Engine': '2.1 л / 55 л.с. / Бензин', 'vehicleTransmission': 'механическая', 'Привод': 'полный', 'Руль': 'Левый', 'Состояние': 'Не требует ремонта', 'Владельцы': '3 или более', 'ПТС': 'Оригинал', 'Владение': '10 лет и 9 месяцев', 'Таможня': 'Растаможен'}
ad {'car_url': 'https://auto.ru/cars/used/sale/gaz/69/1103065284-002e8df5/', 'parsing_unixtime': 1622017475, 'complectation_d

ad {'car_url': 'https://auto.ru/cars/used/sale/gaz/21/1103680785-395a4c1d/', 'parsing_unixtime': 1622017480, 'complectation_dict': '{"id":"0"}', 'equipment_dict': '{}', 'mileage': '65001', 'model_info': '{"code":"21","name":"21 «Волга»","ru_name":"21 «Волга»","morphology":{}', 'model_name': '21', 'super_gen': '{"id":"7867811","year_from":1956,"year_to":1970,"price_segment":"MEDIUM"}', 'vendor': 'RUSSIAN', 'title': 'ГАЗ 21 «Волга»  21Р', 'price': '350\xa0000\xa0₽', 'productionDate': '1969', 'Пробег': '65 001 км', 'bodyType': 'седан', 'color': 'серый', 'Engine': '2.5 л / 75 л.с. / Бензин', 'vehicleTransmission': 'механическая', 'Привод': 'задний', 'Руль': 'Левый', 'Состояние': 'Не требует ремонта', 'Владельцы': '2 владельца', 'ПТС': 'Оригинал', 'Таможня': 'Растаможен'}
ad {'car_url': 'https://auto.ru/cars/used/sale/zaz/966/1103424642-70cfdb52/', 'parsing_unixtime': 1622017480, 'complectation_dict': '{"id":"0"}', 'equipment_dict': '{}', 'mileage': '34000', 'model_info': '{"code":"966","na

ad {'car_url': 'https://auto.ru/cars/used/sale/gaz/21/1103532491-5fb1add7/', 'parsing_unixtime': 1622017483, 'complectation_dict': '{"id":"0"}', 'equipment_dict': '{}', 'mileage': '14000', 'model_info': '{"code":"21","name":"21 «Волга»","ru_name":"21 «Волга»","morphology":{}', 'model_name': '21', 'super_gen': '{"id":"7867811","year_from":1956,"year_to":1970,"price_segment":"MEDIUM"}', 'vendor': 'RUSSIAN', 'title': 'ГАЗ 21 «Волга»  21Р', 'price': '100\xa0000\xa0₽', 'productionDate': '1969', 'Пробег': '14 000 км', 'bodyType': 'седан', 'color': 'голубой', 'Engine': '2.5 л / 75 л.с. / Бензин', 'vehicleTransmission': 'механическая', 'Привод': 'задний', 'Руль': 'Левый', 'Состояние': 'Не требует ремонта', 'Владельцы': '1 владелец', 'ПТС': 'Оригинал', 'Таможня': 'Растаможен'}
ad {'car_url': 'https://auto.ru/cars/used/sale/gaz/21/1100387694-32d302ec/', 'parsing_unixtime': 1622017483, 'complectation_dict': '{"id":"0"}', 'equipment_dict': '{}', 'mileage': '1', 'model_info': '{"code":"21","name":"

In [9]:
file

'_auto_ru_moscow_1970-1980_50_prev.csv'

In [32]:
import os
import pandas as pd
import numpy as np
import re


df = pd.DataFrame()              # инициализируем датафрейм
list_file = os.listdir(path=".") # получаем список файлов в папке


for file in list_file:
    if '.csv' in file: # если у файла расширение .csv, то читаем его как датафрейм
        df_temp = pd.read_csv(file)
        df = pd.concat([df, df_temp], ignore_index=True)
        
df = df.drop_duplicates(subset=['car_url']) # удаляем дубликаты по URL объявления

df = df.replace('{}',  None) # заменяем пустые словари на NaN

# Предварительная обработка некоторых признаков
df['mileage'] = df['mileage'].apply(lambda x: int(x) if type(x) == str else x) # приводим 'mileage' к числовму виду

df['fuelType'] = df['Engine'].apply(
    lambda x: x.split('/')[2] if type(x) == str else x)

df['enginePower'] = np.where(df['fuelType'] == ' Электро', df['Engine'].apply(
    lambda x: x.split('/')[0] if type(x) == str else x), df['Engine'].apply(
    lambda x: x.split('/')[1] if type(x) == str else x))

df['engineDisplacement'] = np.where(df['fuelType'] == ' Электро', df['Engine'].apply(
    lambda x: x.split('/')[1] if type(x) == str else x), df['Engine'].apply(
    lambda x: x.split('/')[0] if type(x) == str else x))
    
df['brand'] =df['title'].apply(lambda x: x.split()[0] if type(x) == str else x)
df['numberOfDoors'] = df['bodyType'].str.extract('(\d+)').fillna(4).astype(int)

df['price'] = df['price'].str.replace('\xa0', '')
df['price'] = df['price'].str.replace('₽', '')
df.dropna(subset=['price'], inplace=True)
df['price'] = df['price'].str.replace('от ', '').astype(int)

df = df.replace(np.nan,  None)

df.to_csv('auto_ru_26_05_2021.csv', encoding = 'utf-8', index=False) # записываем в общий файл для использования на каггл

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68518 entries, 0 to 93662
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Engine               68518 non-null  object 
 1   bodyType             68518 non-null  object 
 2   car_url              68518 non-null  object 
 3   color                68518 non-null  object 
 4   complectation_dict   68518 non-null  object 
 5   equipment_dict       68518 non-null  object 
 6   mileage              68518 non-null  float64
 7   model_info           68518 non-null  object 
 8   model_name           68518 non-null  object 
 9   parsing_unixtime     68518 non-null  float64
 10  price                68518 non-null  int32  
 11  productionDate       68518 non-null  float64
 12  super_gen            68518 non-null  object 
 13  title                68518 non-null  object 
 14  vehicleTransmission  68518 non-null  object 
 15  vendor               68518 non-null 

In [22]:
df.price.unique()

array(['3900000', '1490000', '395000', ..., 'от 4005221', 'от 1505900',
       'от 617500'], dtype=object)

In [70]:
df.head()

,Engine,bodyType,car_url,color,complectation_dict,equipment_dict,model_info,model_name,parsing_unixtime,price,...,Руль,Состояние,Таможня,mileage,Владение,fuelType,enginePower,engineDisplacement,brand,numberOfDoors
0,2.0 л / 131 л.с. / Бензин,седан,https://auto.ru/cars/used/sale/ford/focus_na/1...,зелёный,"{""id"":""0""}",{},"{""code"":""FOCUS_NA"",""name"":""Focus (North Americ...",FOCUS_NA,1.621606e+09,230 000 ₽,...,Левый,Не требует ремонта,Растаможен,NaN,NaN,Бензин,131 л.с.,2.0 л,Ford,4
1,1.8 л / 125 л.с. / Бензин,седан,https://auto.ru/cars/used/sale/volkswagen/pass...,зелёный,"{""id"":""0""}","{""alloy-wheel-disks"":true,""tinted-glass"":true,...","{""code"":""PASSAT"",""name"":""Passat"",""ru_name"":""Па...",PASSAT,1.621606e+09,190 000 ₽,...,Левый,Не требует ремонта,Растаможен,422189.0,10 месяцев,Бензин,125 л.с.,1.8 л,Volkswagen,4
2,2.8 л / 193 л.с. / Бензин,седан,https://auto.ru/cars/used/sale/bmw/3er/1103499...,зелёный,"{""id"":""0""}","{""cruise-control"":true,""tinted-glass"":true,""es...","{""code"":""3ER"",""name"":""3 серии"",""ru_name"":""3 се...",3ER,1.621606e+09,535 000 ₽,...,Левый,Не требует ремонта,Растаможен,205000.0,10 месяцев,Бензин,193 л.с.,2.8 л,BMW,4
3,1.3 л / 80 л.с. / Бензин,внедорожник 3 дв.,https://auto.ru/cars/used/sale/suzuki/jimny/11...,серебристый,"{""id"":""0""}","{""airbag-driver"":true,""aux"":true,""usb"":true,""e...","{""code"":""JIMNY"",""name"":""Jimny"",""ru_name"":""Джим...",JIMNY,1.621606e+09,280 000 ₽,...,Левый,Не требует ремонта,Растаможен,205000.0,6 лет и 2 месяца,Бензин,80 л.с.,1.3 л,Suzuki,3
4,1.7 л / 79 л.с. / Бензин,внедорожник 3 дв.,https://auto.ru/cars/used/sale/vaz/2121/110349...,белый,"{""id"":""0""}","{""airbag-driver"":true,""aux"":true,""usb"":true,""e...","{""code"":""2121"",""name"":""2121 (4x4)"",""ru_name"":""...",2121,1.621606e+09,от 84 000 ₽,...,Левый,Не требует ремонта,Растаможен,89922.0,6 лет и 2 месяца,Бензин,79 л.с.,1.7 л,LADA,3


In [71]:
df.iloc[500]

Engine                                         1.8 л / 182 л.с. / Бензин
bodyType                                                   хэтчбек 3 дв.
car_url                https://auto.ru/cars/used/sale/toyota/celica/1...
color                                                            красный
complectation_dict                                            {"id":"0"}
equipment_dict         {"tinted-glass":true,"airbag-driver":true,"aux...
model_info             {"code":"CELICA","name":"Celica","ru_name":"Се...
model_name                                                        CELICA
parsing_unixtime                                            1621605731.0
price                                                          550 000 ₽
productionDate                                                      1999
super_gen              {"id":"4995560","name":"VII (T230)","ru_name":...
title                                           Toyota Celica VII (T230)
vehicleTransmission                                